In [1]:
import numpy as np
import pandas as pd

# Read data

In [2]:
# Read data from csv
df = pd.read_csv("data/rental_prices_singapore_preprocessed.csv")

In [3]:
# Show dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1576 entries, 0 to 1575
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             1576 non-null   int64  
 1   size              1576 non-null   int64  
 2   bedrooms          1576 non-null   object 
 3   bathrooms         1576 non-null   int64  
 4   latitude          1576 non-null   float64
 5   longitude         1576 non-null   float64
 6   meters_to_school  1576 non-null   int64  
 7   property_type     1576 non-null   object 
 8   furnishing        1576 non-null   object 
 9   year              1576 non-null   int64  
 10  meters_to_mrt     1576 non-null   int64  
 11  high_floor        1576 non-null   bool   
 12  new               1576 non-null   bool   
 13  renovated         1576 non-null   bool   
 14  view              1576 non-null   bool   
 15  penthouse         1576 non-null   bool   
dtypes: bool(5), float64(2), int64(6), object(3

In [4]:
# Show top five rows
df.head()

,price,size,bedrooms,bathrooms,latitude,longitude,meters_to_school,property_type,furnishing,year,meters_to_mrt,high_floor,new,renovated,view,penthouse
0,3000,400,1,1,1.312952,103.887868,422,Apartment,Fully Furnished,2012,450,False,False,False,False,False
1,2000,1130,Room,1,1.328820,103.912904,3573,Apartment,Fully Furnished,2012,810,False,False,False,False,False
2,7400,3800,5,4,1.389444,103.857002,568,Apartment,Fully Furnished,2012,450,False,False,False,False,False
3,1000,120,Room,1,1.429261,103.828917,1090,Apartment,Fully Furnished,2012,700,False,False,False,False,False
4,4300,689,1,1,1.297356,103.836707,1262,Apartment,Fully Furnished,2012,420,False,False,False,False,False
